In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import models
from torchvision.models.vgg import VGG
from sklearn.metrics import confusion_matrix
import pandas as pd
import scipy.misc
import random
import sys

if '/opt/ros/kinetic/lib/python2.7/dist-packages' in sys.path:
    sys.path.remove('/opt/ros/kinetic/lib/python2.7/dist-packages')
import cv2

from torch.optim import lr_scheduler
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader

from matplotlib import pyplot as plt
import numpy as np
import time
import os

## Define FCN16s model for deconvolution layers

In [2]:
class FCN16s(nn.Module):

    def __init__(self, pretrained_net, n_class):
        super(FCN16s, self).__init__()
        self.n_class = n_class
        self.pretrained_net = pretrained_net
        self.relu    = nn.ReLU(inplace = True)
        self.deconv1 = nn.ConvTranspose2d(512, 512, kernel_size=3, stride=2, padding=1, dilation=1, output_padding=1)
        self.bn1     = nn.BatchNorm2d(512)
        self.deconv2 = nn.ConvTranspose2d(512, 256, kernel_size=3, stride=2, padding=1, dilation=1, output_padding=1)
        self.bn2     = nn.BatchNorm2d(256)
        self.deconv3 = nn.ConvTranspose2d(256, 128, kernel_size=3, stride=2, padding=1, dilation=1, output_padding=1)
        self.bn3     = nn.BatchNorm2d(128)
        self.deconv4 = nn.ConvTranspose2d(128, 64, kernel_size=3, stride=2, padding=1, dilation=1, output_padding=1)
        self.bn4     = nn.BatchNorm2d(64)
        self.deconv5 = nn.ConvTranspose2d(64, 32, kernel_size=3, stride=2, padding=1, dilation=1, output_padding=1)
        self.bn5     = nn.BatchNorm2d(32)
        self.classifier = nn.Conv2d(32, n_class, kernel_size=1)

    def forward(self, x):
        output = self.pretrained_net(x)
        x5 = output['x5']  # size=(N, 512, x.H/32, x.W/32)
        x4 = output['x4']  # size=(N, 512, x.H/16, x.W/16)

        score = self.relu(self.deconv1(x5))               # size=(N, 512, x.H/16, x.W/16)
        score = self.bn1(score + x4)                      # element-wise add, size=(N, 512, x.H/16, x.W/16)
        score = self.bn2(self.relu(self.deconv2(score)))  # size=(N, 256, x.H/8, x.W/8)
        score = self.bn3(self.relu(self.deconv3(score)))  # size=(N, 128, x.H/4, x.W/4)
        score = self.bn4(self.relu(self.deconv4(score)))  # size=(N, 64, x.H/2, x.W/2)
        score = self.bn5(self.relu(self.deconv5(score)))  # size=(N, 32, x.H, x.W)
        score = self.classifier(score)                    # size=(N, n_class, x.H/1, x.W/1)
        
        return score

## Define VGG16 for convolution layers

In [3]:
class VGGNet(VGG):
    def __init__(self, pretrained=True, model='vgg16', requires_grad=True, remove_fc=True, show_params=False):
        super(VGGNet, self).__init__(make_layers(cfg[model]))
        self.ranges = ranges[model]

        if pretrained:
            exec("self.load_state_dict(models.%s(pretrained=True).state_dict())" % model)

        if not requires_grad:
            for param in super().parameters():
                param.requires_grad = False

        if remove_fc:  # delete redundant fully-connected layer params, can save memory
            del self.classifier

        if show_params:
            for name, param in self.named_parameters():
                print(name, param.size())

    def forward(self, x):
        output = {}

        # get the output of each maxpooling layer (5 maxpool in VGG net)
        for idx in range(len(self.ranges)):
            for layer in range(self.ranges[idx][0], self.ranges[idx][1]):      
                x = self.features[layer](x)
            output["x%d"%(idx+1)] = x
        return output

In [4]:
ranges = {
    'vgg11': ((0, 3), (3, 6),  (6, 11),  (11, 16), (16, 21)),
    'vgg13': ((0, 5), (5, 10), (10, 15), (15, 20), (20, 25)),
    'vgg16': ((0, 5), (5, 10), (10, 17), (17, 24), (24, 31)),
    'vgg19': ((0, 5), (5, 10), (10, 19), (19, 28), (28, 37))
}

# cropped version from https://github.com/pytorch/vision/blob/master/torchvision/models/vgg.py
cfg = {
    'vgg11': [64, 'M', 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
    'vgg13': [64, 64, 'M', 128, 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
    'vgg16': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 'M', 512, 512, 512, 'M', 512, 512, 512, 'M'],
    'vgg19': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 256, 'M', 512, 512, 512, 512, 'M', 512, 512, 512, 512, 'M'],
}

def make_layers(cfg, batch_norm=False):
    layers = []
    in_channels = 3
    for v in cfg:
        if v == 'M':
            layers += [nn.MaxPool2d(kernel_size=2, stride=2)]
        else:
            conv2d = nn.Conv2d(in_channels, v, kernel_size=3, padding=1)
            if batch_norm:
                layers += [conv2d, nn.BatchNorm2d(v), nn.ReLU(inplace=True)]
            else:
                layers += [conv2d, nn.ReLU(inplace=True)]
            in_channels = v
    return nn.Sequential(*layers)

## Define training parameter

In [5]:
batch_size = 4
epochs     = 10  #500
lr         = 1e-4
momentum   = 0
w_decay    = 1e-5
step_size  = 50
gamma      = 0.5
model_use  = "ds" #"domain_randomization"#"ds_final"#"style_transfer"#"virtual" # "products_20
n_class = 21
class_list = ["3m", "andes", "cocacola", "crayola", "folgers", "kellogg", "hunts", "heineken", "kleenex", "libava", "macadamia", "milo", "mm", "pocky", "stax", "swissmiss", "vanish", "viva", "raisins", "kotex"]
# class_list = ["3m", "andes", "cocacola", "crayola", "folgers", "kellogg", "milo", "stax", "swissmiss", "viva"]

## Define path, directory trainning environment

In [6]:
# get data
data_dir  = os.path.join("data", model_use)
if not os.path.exists(data_dir):
    print("Data not found!")
# create dir for model
model_dir = os.path.join("models", model_use)
if not os.path.exists(model_dir):
    os.makedirs(model_dir)
# create dir for score
score_dir = os.path.join("scores", model_use)
if not os.path.exists(score_dir):
    os.makedirs(score_dir)

use_gpu = torch.cuda.is_available()
num_gpu = list(range(torch.cuda.device_count()))

vgg_model = VGGNet(requires_grad=True, remove_fc=True)
fcn_model = FCN16s(pretrained_net=vgg_model, n_class=n_class)
#use_gpu = False
if use_gpu:
    ts = time.time()
    vgg_model = vgg_model.cuda()
    fcn_model = fcn_model.cuda()
    fcn_model = nn.DataParallel(fcn_model, device_ids=num_gpu)
    print("Finish cuda loading, time elapsed {}".format(time.time() - ts))
    
fcn_model.load_state_dict(torch.load("/home/arg/pytorch_fcn/models/ds/virtual.pkl"))
# fcn_model.load_state_dict(torch.load("/home/arg/pytorch_fcn/models/ds/style.pkl"))
# fcn_model.load_state_dict(torch.load("/home/arg/sis_lab_all/12-1_Pytorch-FCN/models/ds_final/style_transfer.pkl"))


Finish cuda loading, time elapsed 3.31510615349


<All keys matched successfully>

## Visualize model

In [7]:
print(fcn_model)
params = list(fcn_model.parameters())

DataParallel(
  (module): FCN16s(
    (pretrained_net): VGGNet(
      (features): Sequential(
        (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): ReLU(inplace=True)
        (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (3): ReLU(inplace=True)
        (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
        (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (6): ReLU(inplace=True)
        (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (8): ReLU(inplace=True)
        (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
        (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (11): ReLU(inplace=True)
        (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (13): ReLU(inplace=True)
        (14): Conv2d(256, 256, kernel_size=(3, 3), str

Dataset class
-------------

``torch.utils.data.Dataset`` is an abstract class representing a
dataset.
Your custom dataset should inherit ``Dataset`` and override the following
methods:

-  ``__len__`` so that ``len(dataset)`` returns the size of the dataset.
-  ``__getitem__`` to support the indexing such that ``dataset[i]`` can
   be used to get $i$\ th sample

Let's create a dataset class for our face landmarks dataset. We will
read the csv in ``__init__`` but leave the reading of images to
``__getitem__``. This is memory efficient because all the images are not
stored in the memory at once but read as required.



In [8]:
means     = np.array([103.939, 116.779, 123.68]) / 255. # mean of three channels in the order of BGR
h, w      = 480, 640
val_h     = h
val_w     = w
class product_dataset(Dataset):

    def __init__(self, csv_file, phase, n_class=n_class, flip_rate=0.):
        self.data      = pd.read_csv(csv_file)
        self.means     = means
        self.n_class   = n_class
        self.flip_rate = flip_rate
        if phase == 'train':
            self.flip_rate = 0.5

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_name   = self.data.iloc[idx, 0]
        img        = cv2.imread(os.path.join(data_dir, img_name),cv2.IMREAD_UNCHANGED)
        label_name = self.data.iloc[idx, 1]
        label      = cv2.imread(os.path.join(data_dir, label_name), cv2.IMREAD_GRAYSCALE)
        origin_img = img

        if random.random() < self.flip_rate:
            img   = np.fliplr(img)
            label = np.fliplr(label)

        # reduce mean
#         print(data_dir + img_name)
#         plt.figure()
#         plt.imshow(img)
#         plt.show()
        img = img[:, :, ::-1]  # switch to BGR
        
        img = np.transpose(img, (2, 0, 1)) / 255.
        img[0] -= self.means[0]
        img[1] -= self.means[1]
        img[2] -= self.means[2]

        # convert to tensor
        img = torch.from_numpy(img.copy()).float()
        label = torch.from_numpy(label.copy()).long()

        # create one-hot encoding
        h, w = label.size()
        target = torch.zeros(self.n_class, h, w)
        
        for index, obj in enumerate(class_list):
            if obj in img_name:
#                 print index+1, obj
                target[index+1][label == 255] = 1
                label[label == 255] = index+1

        target[0][label == 0] = 1
 
        sample = {'X': img, 'Y': target, 'l': label, 'origin': origin_img}

        return sample

## Define dataloader and optimizer

In [9]:
# initial dataloader for trainning and validation
train_file = os.path.join(data_dir, "train_real_10.csv")
val_file   = os.path.join(data_dir, "val_real.csv")
train_data = product_dataset(csv_file = train_file, phase = 'train')
val_data   = product_dataset(csv_file = val_file, phase = 'val', flip_rate = 0)
dataloader = DataLoader(train_data, batch_size = batch_size, shuffle=True, num_workers = 0)
val_loader = DataLoader(val_data, batch_size = 5, num_workers = 0)

dataiter = iter(val_loader)

# define loss function
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.RMSprop(fcn_model.parameters(), lr = lr, momentum = momentum, weight_decay = w_decay)
# decay LR by a factor of 0.5 every step_size = 50 epochs
scheduler = lr_scheduler.StepLR(optimizer, step_size = step_size, gamma = gamma)  

## Train

In [10]:
def train():
    for epoch in range(epochs):
        fcn_model.train()
        scheduler.step()
#         configs    = "FCNs_{}_batch{}_epoch{}_RMSprop_lr{}"\
#             .format(model_use, batch_size, epoch, lr)
        configs    = "FCNs_{}_epoch{}".format(model_use, epoch)
        model_path = os.path.join(model_dir, configs)
        
        ts = time.time()
        for iter, batch in enumerate(dataloader):
            optimizer.zero_grad()

            if use_gpu:
                inputs = Variable(batch['X'].cuda())
                labels = Variable(batch['Y'].cuda())
            else:
                inputs, labels = Variable(batch['X']), Variable(batch['Y'])

            outputs = fcn_model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            if iter % 10 == 0:
                print("epoch{}, iter{}, loss: {}".format(epoch+1, iter, loss.item()))
                
#             if iter % 2000 == 0:
#                 torch.save(fcn_model.state_dict(),model_path + '_'+str(iter)+'.pkl')
        
        print("Finish epoch {}, time elapsed {}".format(epoch, time.time() - ts))
        torch.save(fcn_model.state_dict(),model_path + '_epoch' + str(epoch) + '.pkl')
#         if epoch % 20 == 0:
#             torch.save(fcn_model.state_dict(),model_path + '.pkl')

        val(epoch)

## Evaluation

In [11]:
def val(epoch):
    fcn_model.eval()
    TP = np.zeros(n_class-1)
    FN = np.zeros(n_class-1)
    FP = np.zeros(n_class-1)
    total_ious = []
    pixel_accs = []
    for iter, batch in enumerate(val_loader):
        if use_gpu:
            inputs = Variable(batch['X'].cuda())
        else:
            inputs = Variable(batch['X'])

        output = fcn_model(inputs)
        output = output.data.cpu().numpy()

        N, _, h, w = output.shape
        pred = output.transpose(0, 2, 3, 1).reshape(-1, n_class).argmax(axis=1).reshape(N, h, w)

        target = batch['l'].cpu().numpy().reshape(N, h, w)
        for p, t in zip(pred, target):
            pixel_accs.append(pixel_acc(p, t))
            _TP, _FN, _FP =  analysis(p, t, h, w)
            TP += _TP[1:n_class]
            FN += _FN[1:n_class]
            FP += _FP[1:n_class]
            
    recall = TP / (TP + FN)
    precision = TP / (TP + FP)
    ious = TP / (TP + FN + FP)
    fscore = 2*TP / (2*TP + FN + FP)
    total_ious = np.array(total_ious).T  # n_class * val_len
    pixel_accs = np.array(pixel_accs).mean()
    
    np.set_printoptions(formatter=dict(float=lambda t:"%6.4f" % t))
    print("epoch{}, pix_acc: {}, meanIoU: {}, IoUs: {}, recall: {}, precision: {}, fscore: {}"\
          .format(epoch, pixel_accs, np.nanmean(ious), ious, recall, precision, fscore))
    
    f1 = open(score_dir + "/cls_acc_log.txt","a+")
    f1.write('epoch:'+ str(epoch) + ', pix_acc: ' + str(pixel_accs) + '\n' )
    f2 = open(score_dir + "/cls_iou_log.txt","a+")
    f2.write('epoch:'+ str(epoch) + ', class ious: ' + str(ious) + '\n' )
    f3 = open(score_dir + "/mean_iou_log.txt","a+")
    f3.write('epoch:'+ str(epoch) + ', mean IoU: ' + str(np.nanmean(ious)) + '\n' ) 
    f4 = open(score_dir + "/recall_log.txt","a+")
    f4.write('epoch:'+ str(epoch) + ', class recall: ' + str(recall) + '\n' )
    f5 = open(score_dir + "/precision_log.txt","a+")
    f5.write('epoch:'+ str(epoch) + ', class precision: ' + str(precision) + '\n' )    
    f6 = open(score_dir + "/fscore_log.txt","a+")
    f6.write('epoch:'+ str(epoch) + ', class fscore: ' + str(fscore) + '\n' )  
    

def analysis(pred, target, h, w):
    # TP, FN, FP, TN
    TP = np.zeros(n_class)
    FN = np.zeros(n_class)
    FP = np.zeros(n_class)

    target = target.reshape(h * w)
    pred = pred.reshape(h * w)

    con_matrix = confusion_matrix(target, pred,labels = np.arange(0,n_class,1))
    con_matrix[0][0] = 0
    for i in range(0, n_class):
        for j in range(0, n_class):
            if i == j:
                TP[i] += con_matrix[i][j]
            if i != j:
                FP[j] += con_matrix[i][j]
                FN[i] += con_matrix[i][j]
    return TP, FN, FP
                
def pixel_acc(pred, target):
    correct = (pred == target).sum()
    total   = (target == target).sum()
    return float(correct)/float(total)
#     return correct / total

## Prediction Result 

In [12]:
def prediction(model_name):
    
    # load pretrain models
    state_dict = torch.load(os.path.join(model_dir, model_name))
    fcn_model.load_state_dict(state_dict)
    
    batch = dataiter.next()
    if use_gpu:
        inputs = Variable(batch['X'].cuda())
    else:
        inputs = Variable(batch['X'])
    img    = batch['origin'] 
    label  = batch['l']
    
    output = fcn_model(inputs)
    output = output.data.cpu().numpy()

    N, _, h, w = output.shape
    pred = output.transpose(0, 2, 3, 1).reshape(-1, n_class).argmax(axis = 1).reshape(N, h, w)

    # show images
    plt.figure(figsize = (10, 12))
    img = img.numpy()
    for i in range(N):
        img[i] = cv2.cvtColor(img[i], cv2.COLOR_BGR2RGB)
        plt.subplot(N, 3, i*3 + 1)
        plt.title("origin_img")
        plt.imshow(img[i])
        #print(np.unique(_img[i]))

        plt.subplot(N, 3, i*3 + 2)
        plt.title("label_img")
        plt.imshow(label[i],cmap = "brg",vmin = 0, vmax = n_class - 1)

        plt.subplot(N, 3, i*3 + 3)
        plt.title("prediction")
        plt.imshow(pred[i],cmap = "brg",vmin = 0, vmax = n_class - 1)

    plt.show()

In [13]:
train()

/home/arg/.local/lib/python2.7/site-packages/torch/optim/lr_scheduler.py:100: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule.See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


epoch1, iter0, loss: 0.0386433750391
epoch1, iter10, loss: 0.0143192252144
epoch1, iter20, loss: 0.0176094584167
epoch1, iter30, loss: 0.0131550207734
epoch1, iter40, loss: 0.0129877692088
epoch1, iter50, loss: 0.0214122068137
epoch1, iter60, loss: 0.0132346088067
epoch1, iter70, loss: 0.00762546202168
epoch1, iter80, loss: 0.0143702523783
epoch1, iter90, loss: 0.0191653240472
epoch1, iter100, loss: 0.0171561576426
epoch1, iter110, loss: 0.0086773512885
epoch1, iter120, loss: 0.00865317601711
epoch1, iter130, loss: 0.0231681577861
epoch1, iter140, loss: 0.0310554131866
epoch1, iter150, loss: 0.0198038816452
epoch1, iter160, loss: 0.0109806060791
epoch1, iter170, loss: 0.0145753966644
epoch1, iter180, loss: 0.0146537739784
epoch1, iter190, loss: 0.0127118807286
epoch1, iter200, loss: 0.0125248050317
epoch1, iter210, loss: 0.00450734375045
epoch1, iter220, loss: 0.0159170422703
epoch1, iter230, loss: 0.00900078285486
epoch1, iter240, loss: 0.0082348510623
epoch1, iter250, loss: 0.0251687

epoch3, iter280, loss: 0.00144000875298
epoch3, iter290, loss: 0.0144307194278
epoch3, iter300, loss: 0.0061675622128
epoch3, iter310, loss: 0.00361498584971
epoch3, iter320, loss: 0.00623723678291
epoch3, iter330, loss: 0.00355886737816
epoch3, iter340, loss: 0.00263846013695
epoch3, iter350, loss: 0.00291771860793
epoch3, iter360, loss: 0.0022816832643
epoch3, iter370, loss: 0.00427305744961
epoch3, iter380, loss: 0.00384113634937
epoch3, iter390, loss: 0.00165507232305
epoch3, iter400, loss: 0.00253339600749
epoch3, iter410, loss: 0.00212250370532
epoch3, iter420, loss: 0.00258925533853
epoch3, iter430, loss: 0.00145778409205
epoch3, iter440, loss: 0.0106456167996
epoch3, iter450, loss: 0.00309961428866
epoch3, iter460, loss: 0.00345535506494
epoch3, iter470, loss: 0.00524043943733
epoch3, iter480, loss: 0.00959899276495
epoch3, iter490, loss: 0.00298907514662
epoch3, iter500, loss: 0.00910391658545
epoch3, iter510, loss: 0.0092060258612
epoch3, iter520, loss: 0.0152346454561
epoch3

epoch5, iter550, loss: 0.0138836298138
epoch5, iter560, loss: 0.00128083210438
epoch5, iter570, loss: 0.0038120073732
epoch5, iter580, loss: 0.002252911916
epoch5, iter590, loss: 0.00487257027999
epoch5, iter600, loss: 0.0153174642473
epoch5, iter610, loss: 0.00241296109743
epoch5, iter620, loss: 0.00387494941242
epoch5, iter630, loss: 0.00306069501676
epoch5, iter640, loss: 0.00329984677956
epoch5, iter650, loss: 0.00133404275402
epoch5, iter660, loss: 0.00654144724831
epoch5, iter670, loss: 0.00829073973
epoch5, iter680, loss: 0.00129140540957
epoch5, iter690, loss: 0.000693718728144
epoch5, iter700, loss: 0.00153285462875
epoch5, iter710, loss: 0.00135851872619
Finish epoch 4, time elapsed 303.335637093
epoch4, pix_acc: 0.980741865406, meanIoU: 0.723552617948, IoUs: [0.7216 0.3845 0.6561 0.8444 0.8275 0.6778 0.4227 0.7334 0.8569 0.7680
 0.8030 0.8048 0.8091 0.8466 0.8572 0.8983 0.7240 0.5519 0.7039 0.5792], recall: [0.7843 0.4016 0.6708 0.9136 0.8648 0.7042 0.4329 0.9278 0.9531 0.83

epoch8, iter0, loss: 0.00220658094622
epoch8, iter10, loss: 0.00187102600466
epoch8, iter20, loss: 0.0127142472193
epoch8, iter30, loss: 0.00218382664025
epoch8, iter40, loss: 0.00189127260819
epoch8, iter50, loss: 0.00215331534855
epoch8, iter60, loss: 0.00117677263916
epoch8, iter70, loss: 0.00252917525358
epoch8, iter80, loss: 0.00121619715355
epoch8, iter90, loss: 0.00250171124935
epoch8, iter100, loss: 0.00137780047953
epoch8, iter110, loss: 0.00129525968805
epoch8, iter120, loss: 0.00112448749132
epoch8, iter130, loss: 0.00136910565197
epoch8, iter140, loss: 0.00624791579321
epoch8, iter150, loss: 0.00142387999222
epoch8, iter160, loss: 0.00325898313895
epoch8, iter170, loss: 0.00112653360702
epoch8, iter180, loss: 0.00145175226498
epoch8, iter190, loss: 0.0022593990434
epoch8, iter200, loss: 0.0013337720884
epoch8, iter210, loss: 0.00204028747976
epoch8, iter220, loss: 0.00128184084315
epoch8, iter230, loss: 0.00385282165371
epoch8, iter240, loss: 0.00259982678108
epoch8, iter25

epoch10, iter270, loss: 0.0026807480026
epoch10, iter280, loss: 0.00262762000784
epoch10, iter290, loss: 0.00102209602483
epoch10, iter300, loss: 0.00359611283056
epoch10, iter310, loss: 0.00126878125593
epoch10, iter320, loss: 0.00146458216477
epoch10, iter330, loss: 0.00132545712404
epoch10, iter340, loss: 0.00180875312071
epoch10, iter350, loss: 0.00216083158739
epoch10, iter360, loss: 0.00159764604177
epoch10, iter370, loss: 0.00140639930032
epoch10, iter380, loss: 0.00108785787597
epoch10, iter390, loss: 0.0121378358454
epoch10, iter400, loss: 0.000962217396591
epoch10, iter410, loss: 0.00199215649627
epoch10, iter420, loss: 0.0144368736073
epoch10, iter430, loss: 0.00170910020825
epoch10, iter440, loss: 0.00199682358652
epoch10, iter450, loss: 0.00429882109165
epoch10, iter460, loss: 0.000957680458669
epoch10, iter470, loss: 0.000979012693278
epoch10, iter480, loss: 0.00126964575611
epoch10, iter490, loss: 0.0016465683002
epoch10, iter500, loss: 0.00121052342001
epoch10, iter510,

In [14]:
fcn_model.load_state_dict(torch.load("/home/arg/pytorch_fcn/models/ds/style.pkl"))
train()

epoch1, iter0, loss: 0.0563288070261
epoch1, iter10, loss: 0.0224344339222
epoch1, iter20, loss: 0.0221008881927
epoch1, iter30, loss: 0.00708737364039
epoch1, iter40, loss: 0.0163025986403
epoch1, iter50, loss: 0.014272400178
epoch1, iter60, loss: 0.00992776174098
epoch1, iter70, loss: 0.00511758495122
epoch1, iter80, loss: 0.024222208187
epoch1, iter90, loss: 0.0121217146516
epoch1, iter100, loss: 0.00465448433533
epoch1, iter110, loss: 0.00318821961991
epoch1, iter120, loss: 0.00474179163575
epoch1, iter130, loss: 0.00451340992004
epoch1, iter140, loss: 0.0123002380133
epoch1, iter150, loss: 0.00865159276873
epoch1, iter160, loss: 0.00523022701964
epoch1, iter170, loss: 0.00528397923335
epoch1, iter180, loss: 0.0134360743687
epoch1, iter190, loss: 0.0156902521849
epoch1, iter200, loss: 0.0139035927132
epoch1, iter210, loss: 0.00440478604287
epoch1, iter220, loss: 0.00593716744334
epoch1, iter230, loss: 0.00640032067895
epoch1, iter240, loss: 0.00415230635554
epoch1, iter250, loss: 0

epoch3, iter280, loss: 0.00145780621096
epoch3, iter290, loss: 0.00109176326077
epoch3, iter300, loss: 0.00351006747223
epoch3, iter310, loss: 0.00175159785431
epoch3, iter320, loss: 0.00462011294439
epoch3, iter330, loss: 0.00234081386589
epoch3, iter340, loss: 0.00176623626612
epoch3, iter350, loss: 0.00266714044847
epoch3, iter360, loss: 0.00190440029837
epoch3, iter370, loss: 0.00245703454129
epoch3, iter380, loss: 0.00194424577057
epoch3, iter390, loss: 0.00204485980794
epoch3, iter400, loss: 0.00112432485912
epoch3, iter410, loss: 0.00205363566056
epoch3, iter420, loss: 0.00321929622442
epoch3, iter430, loss: 0.00444630905986
epoch3, iter440, loss: 0.00157359847799
epoch3, iter450, loss: 0.00610102666542
epoch3, iter460, loss: 0.00163529801648
epoch3, iter470, loss: 0.00122560025193
epoch3, iter480, loss: 0.00156719656661
epoch3, iter490, loss: 0.00331062613986
epoch3, iter500, loss: 0.00110161560588
epoch3, iter510, loss: 0.00671135168523
epoch3, iter520, loss: 0.00613532867283


epoch5, iter550, loss: 0.00143805635162
epoch5, iter560, loss: 0.00109380017966
epoch5, iter570, loss: 0.00129403744359
epoch5, iter580, loss: 0.00961878523231
epoch5, iter590, loss: 0.00216142693534
epoch5, iter600, loss: 0.00123000063468
epoch5, iter610, loss: 0.00287943240255
epoch5, iter620, loss: 0.00261048926041
epoch5, iter630, loss: 0.00217761006206
epoch5, iter640, loss: 0.0052899774164
epoch5, iter650, loss: 0.00375493708998
epoch5, iter660, loss: 0.0047025885433
epoch5, iter670, loss: 0.00849984306842
epoch5, iter680, loss: 0.00245377002284
epoch5, iter690, loss: 0.005316471681
epoch5, iter700, loss: 0.0148469619453
epoch5, iter710, loss: 0.00148021418136
Finish epoch 4, time elapsed 302.955822945
epoch4, pix_acc: 0.981219989264, meanIoU: 0.757867686663, IoUs: [0.7138 0.5434 0.7173 0.8749 0.6710 0.4522 0.7558 0.8032 0.8781 0.7880
 0.8573 0.8865 0.8152 0.8980 0.8231 0.9044 0.7833 0.5993 0.6027 0.7898], recall: [0.7419 0.5679 0.7478 0.9061 0.7196 0.4558 0.7990 0.8925 0.9477 0.

epoch8, iter0, loss: 0.00242837960832
epoch8, iter10, loss: 0.0032815868035
epoch8, iter20, loss: 0.00112714443821
epoch8, iter30, loss: 0.0108493231237
epoch8, iter40, loss: 0.00318028940819
epoch8, iter50, loss: 0.00199103122577
epoch8, iter60, loss: 0.00179940776434
epoch8, iter70, loss: 0.000787889759522
epoch8, iter80, loss: 0.00341220851988
epoch8, iter90, loss: 0.00150317326188
epoch8, iter100, loss: 0.00172285153531
epoch8, iter110, loss: 0.0011941739358
epoch8, iter120, loss: 0.00302766659297
epoch8, iter130, loss: 0.0016195679782
epoch8, iter140, loss: 0.000633783231024
epoch8, iter150, loss: 0.00420410325751
epoch8, iter160, loss: 0.00140748044942
epoch8, iter170, loss: 0.00191096647177
epoch8, iter180, loss: 0.00231662276201
epoch8, iter190, loss: 0.00202096044086
epoch8, iter200, loss: 0.00204516993836
epoch8, iter210, loss: 0.00114472932182
epoch8, iter220, loss: 0.00132633047178
epoch8, iter230, loss: 0.00782913621515
epoch8, iter240, loss: 0.00343288597651
epoch8, iter2

epoch10, iter260, loss: 0.000816636427771
epoch10, iter270, loss: 0.0010796696879
epoch10, iter280, loss: 0.0016805625055
epoch10, iter290, loss: 0.00151995744091
epoch10, iter300, loss: 0.00388344796374
epoch10, iter310, loss: 0.00130896421615
epoch10, iter320, loss: 0.00102826114744
epoch10, iter330, loss: 0.00374910491519
epoch10, iter340, loss: 0.00268617738038
epoch10, iter350, loss: 0.00246823369525
epoch10, iter360, loss: 0.000756674387958
epoch10, iter370, loss: 0.00401011668146
epoch10, iter380, loss: 0.000840597902425
epoch10, iter390, loss: 0.00394473597407
epoch10, iter400, loss: 0.00280891079456
epoch10, iter410, loss: 0.0132846720517
epoch10, iter420, loss: 0.00281669595279
epoch10, iter430, loss: 0.00217600516044
epoch10, iter440, loss: 0.00735782878473
epoch10, iter450, loss: 0.00156094343401
epoch10, iter460, loss: 0.00936777424067
epoch10, iter470, loss: 0.00110096030403
epoch10, iter480, loss: 0.00174385251012
epoch10, iter490, loss: 0.00365319848061
epoch10, iter500

In [15]:
# prediction("FCNs_ds_epoch9_epoch9.pkl")
# prediction("FCNs_ds_epoch9_epoch9.pkl")
# prediction("FCNs_ds_epoch9_epoch9.pkl")


# # print("real_batch4_epoch9")
# # # prediction("real_batch4_epoch9.pkl")
# # print("virtual_batch4_epoch3")
# # prediction("virtual_batch4_epoch3.pkl")
# # print("style_transfer_batch4_epoch8")
# # prediction("style_transfer_batch4_epoch8.pkl")

# print("real_806")
# prediction("real_806.pkl")
# print("virtual with 806 real image fine-tune")
# prediction("virtual_real_806.pkl")
# print("style with 806 real image fine-tune")
# prediction("style_real_806.pkl")
# print("=================================================================")
# print("real_403")
# prediction("real_403.pkl")
# print("virtual with 403 real image fine-tune")
# prediction("virtual_real_403.pkl")
# print("style with 403 real image fine-tune")
# prediction("style_real_403.pkl")
# print("=================================================================")
# print("real_268")
# prediction("real_268.pkl")
# print("virtual with 268 real image fine-tune")
# prediction("virtual_real_268.pkl")
# print("style with 268 real image fine-tune")
# prediction("style_real_268.pkl")
# # print("style with 268 real image fine-tune")
# # prediction("style_real_268_f.pkl")

# print("=================================================================")
# print("real_201")
# prediction("real_201.pkl")
# print("virtual with 201 real image fine-tune")
# prediction("virtual_real_201.pkl")
# # print("virtual with 201 real image fine-tune")
# # prediction("virtual_real_201_f.pkl")
# print("style with 201 real image fine-tune")
# prediction("style_real_201.pkl")
# # print("style with 201 real image fine-tune")
# # prediction("style_real_201_f.pkl")


In [16]:
def val_prediction(model_name, score_dir_name):
    state_dict = torch.load(os.path.join(model_dir, model_name))
    fcn_model.load_state_dict(state_dict)
    
    fcn_model.eval()
    TP = np.zeros(n_class-1)
    FN = np.zeros(n_class-1)
    FP = np.zeros(n_class-1)
    total_ious = []
    pixel_accs = []
    for iter, batch in enumerate(val_loader):
        if use_gpu:
            inputs = Variable(batch['X'].cuda())
        else:
            inputs = Variable(batch['X'])

        output = fcn_model(inputs)
        output = output.data.cpu().numpy()

        N, _, h, w = output.shape
        pred = output.transpose(0, 2, 3, 1).reshape(-1, n_class).argmax(axis=1).reshape(N, h, w)

        target = batch['l'].cpu().numpy().reshape(N, h, w)
        
        for p, t in zip(pred, target):
            pixel_accs.append(pixel_acc(p, t))
            _TP, _FN, _FP =  analysis(p, t, h, w)
            TP += _TP[1:n_class]
            FN += _FN[1:n_class]
            FP += _FP[1:n_class]

    recall = TP / (TP + FN)
    precision = TP / (TP + FP)
    ious = TP / (TP + FN + FP)
    fscore = 2*TP / (2*TP + FN + FP)
    total_ious = np.array(total_ious).T  # n_class * val_len
    pixel_accs = np.array(pixel_accs).mean()
    
    np.set_printoptions(formatter=dict(float=lambda t:"%6.4f" % t))
    print("model: {}\npix_acc: {}\nmeanIoU: {}\nIoUs:      {}\nrecall:    {}\nprecision: {}\nfscore:    {}".format(model_name,pixel_accs, np.nanmean(ious),ious, recall, precision, fscore))
    print ('             3m     andes   coca  crayo  folger kellog  milo   stax  swiss  viva')
    score_dir_name = score_dir + '/' + score_dir_name
    if not os.path.exists(score_dir_name):
        os.makedirs(score_dir_name)
    
    f1 = open(score_dir_name + "/cls_acc_log.txt","a+")
    f1.write('model: {}, pix_acc: {} \n'.format(model_name, pixel_accs))
    f2 = open(score_dir_name + "/cls_iou_log.txt","a+")
    f2.write('model: {}, class ious: {} \n'.format(model_name, ious))
    f3 = open(score_dir_name + "/mean_iou_log.txt","a+")
    f3.write('model: {}, mean IoU: {} \n'.format(model_name, np.nanmean(ious))) 
    f4 = open(score_dir_name + "/recall_log.txt","a+")
    f4.write('model: {}, class recall: {} \n'.format(model_name, recall))
    f5 = open(score_dir_name + "/precision_log.txt","a+")
    f5.write('model: {}, class precision: {} \n'.format(model_name, precision))    
    f6 = open(score_dir_name + "/fscore_log.txt","a+")
    f6.write('model: {}, class fscore: {} \n'.format(model_name, fscore))  
    

In [17]:
# print("real_806")
# val_prediction("real_806.pkl", "real_806")
# print("\n\nvirtual with 806 real image fine-tune")
# val_prediction("virtual_real_806.pkl", "virtual_real_806")
# print("\n\nstyle with 806 real image fine-tune")
# val_prediction("style_real_806.pkl", "style_real_806")

# print("real_403")
# val_prediction("real_403.pkl", "real_403")
# print("\n\nvirtual with 403 real image fine-tune")
# val_prediction("virtual_real_403.pkl", "virtual_real_403")
# print("\n\nstyle with 403 real image fine-tune")
# val_prediction("style_real_403.pkl", "style_real_403")

# print("real_268")
# val_prediction("real_268.pkl", "real_268")
# print("\n\nvirtual with 268 real image fine-tune")
# val_prediction("virtual_real_268.pkl", "virtual_real_268")
# print("\n\nstyle with 268 real image fine-tune")
# val_prediction("style_real_268.pkl", "style_real_268")
# print("\n\nfail, style with 268 real image fine-tune")
# val_prediction("style_real_268_f.pkl", "style_real_268_f")

# print("real_201")
# val_prediction("real_201.pkl", "real_201")
# print("\n\nvirtual with 201 real image fine-tune")
# val_prediction("virtual_real_201.pkl", "virtual_real_201")
# print("\n\nstyle with 201 real image fine-tune")
# val_prediction("style_real_201.pkl", "style_real_201")
# print("\n\nfail, virtual with 201 real image fine-tune")
# val_prediction("virtual_real_201_f.pkl", "virtual_real_201_f")
# print("\n\nfail, style with 201 real image fine-tune")
# val_prediction("style_real_201_f.pkl", "style_real_201_f")

In [18]:
# print("real_806")
# prediction("real_806.pkl")
# print("virtual with 806 real image fine-tune")
# prediction("virtual_real_806.pkl")
# print("style with 806 real image fine-tune")
# prediction("style_real_806.pkl")
# print("=================================================================")
# print("real_403")
# prediction("real_403.pkl")
# print("virtual with 403 real image fine-tune")
# prediction("virtual_real_403.pkl")
# print("style with 403 real image fine-tune")
# prediction("style_real_403.pkl")
# print("=================================================================")
# print("real_268")
# prediction("real_268.pkl")
# print("virtual with 268 real image fine-tune")
# prediction("virtual_real_268.pkl")
# print("style with 268 real image fine-tune")
# prediction("style_real_268.pkl")
# print("style with 268 real image fine-tune")
# prediction("style_real_268_f.pkl")

# print("=================================================================")
# print("real_201")
# prediction("real_201.pkl")
# print("virtual with 201 real image fine-tune")
# prediction("virtual_real_201.pkl")
# print("virtual with 201 real image fine-tune")
# prediction("virtual_real_201_f.pkl")
# print("style with 201 real image fine-tune")
# prediction("style_real_201.pkl")
# print("style with 201 real image fine-tune")
# prediction("style_real_201_f.pkl")